In [13]:
from flask import Flask
from flask import request, jsonify
from flask_restful import Resource, Api, reqparse
from flask_navigation import Navigation
import blinker
import pandas as pd
import ast
import psycopg2
import pandas.io.sql as sqlio
import requests
import sqlalchemy
from sqlalchemy import create_engine
from pathlib import Path
import json
from multipledispatch import dispatch
from flask import request, render_template
from datetime import date, timedelta
import os
import plotly.graph_objects as go
import plotly.express as px
import kaleido

In [14]:
#cal_df = df_current.copy()
#cal_df["date"] = pd.to_datetime(cal_df["date"])
#cal_df = cal_df.set_index("date")
#cal_df["Education"] = cal_df["Class"]+cal_df["Notes"] + cal_df["Group Study"] + cal_df["Read"]
#cal_df["Education"] = cal_df["Education"].clip(upper=1)

# Helper Functions

In [15]:
def get_properties():
    props = {}
    separator = "="
            
    with open('./database.properties') as file:
        for line in file: 
            if separator in line:
                name, value = line.split(separator, 1)
                props[name.strip()] = value.strip()
    return props

# Databaseclass Database:

In [16]:
class Database:
    def __init__(self):
        db_prop = get_properties()
        self.username = db_prop.get('username')
        self.password = db_prop.get('password')
        self.host = db_prop.get('host')
        self.port = db_prop.get('port')
        self.database = db_prop.get('database')
        #For SQLAlchemy
       
    
    def connect(self):
        connection = psycopg2.connect(user=self.username,
                                      password=self.password,
                                      host=self.host,
                                      port=self.port,
                                      database=self.database)
        return connection, connection.cursor()
    
    def connect_alchemy(self):
        '''Alt library to connect used for importing dataframes'''
        engine = create_engine('postgresql://'+self.username+':'+self.password+'@'+self.host+':'+self.port+'/'+self.database)
        return engine
    
    def execute_query(self, query, values):
        conn, cursor = None, None
        response = ""
        try:
            conn, cursor = self.connect()
            cursor.execute(query, values)
            conn.commit()
            query = "select * from public.task WHERE name = %s and date =%s"
            cursor.execute(query, values)
            conn.commit()
            response = cursor.fetchone()[0]
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            response = "duplicate"
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
            return response
        
    def execute_select(self, query, values):
        conn, cursor = None, None
        response = ""
        
        try:
            conn, cursor = self.connect()
            cursor.execute(query, values)
            conn.commit()
            response = cursor.fetchone()[0]
        except (Exception, psycopg2.Error) as error:
            #Does not exist
            insert_query = 'INSERT INTO public.task ("name", "date", "time") VALUES (%s, %s, %s);'
            response =  self.execute_query(insert_query, values)
            cursor.execute(query, values)
            conn.commit()
            response = cursor.fetchone()[0]
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
            return response
        
        
        
    @dispatch(str, str, str, str, str, bool)
    def insert_food(self, name, date, hour, food_name, size, home_cooked):
        '''
        Check if habit exits
            Insert if it doens't
        Get id of habit
        Insert into food
        '''
        query = "select * from public.task WHERE name = %s and date =%s and time =%s"
        values = (name, date, hour, )
        response = self.execute_select(query, values)#Inserts new value if blank
        
        query = 'INSERT INTO public.food ("name", task_fk, "size", homecooked) VALUES(%s, %s, %s, %s);'
        values = (food_name, int(response), size, home_cooked)
        conn, cursor = None, None
        response = ""
        try:
            conn, cursor = self.connect()
            cursor.execute(query, values)
            conn.commit()
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            response = "duplicate"
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
            return response
    
    @dispatch(str)
    def insert_habit(self, name):
        query = 'INSERT INTO public.task ("name") VALUES (%s);'
        values = tuple(name)
        return self.execute_query(query, values)
    @dispatch(str, str)            
    def insert_habit(self, name, date):
        query = 'INSERT INTO public.task ("name", "date") VALUES (%s, %s);'
        values = (name, date,)
        return self.execute_query(query, values)
    @dispatch(str, str, str)            
    def insert_habit(self, name, date, time):
        query = 'INSERT INTO public.task ("name", "date", "time") VALUES (%s, %s, %s);'
        values = (name, date, time)
        return self.execute_query(query, values)
    
    @dispatch(str)            
    def delete_habit(self, id):
        query = f'DELETE FROM public.task WHERE id = {id};'
        values = (id)
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            cursor.execute(query)
            conn.commit()
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
        
    def read_habits(self):
        query = f"select * from task where date = '{date.today()}'"
        result = None
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            result = sqlio.read_sql_query(query, conn)
        except(Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            if conn:
                cursor.close()
                conn.close()
            return result
        
        
    def read_food_week(self):
        query = f"SELECT  f.name, f.homecooked, t.date, t.time FROM public.food as f INNER JOIN task as t on f.task_fk = t.id where date > '{date.today() - timedelta(days=7)}'"
        result = None
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            result = sqlio.read_sql_query(query, conn)
        except(Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            if conn:
                cursor.close()
                conn.close()
            return result     
        
    def read_habits_week(self):
        query = f"select * from task where date > '{date.today() - timedelta(days=7)}'"
        result = None
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            result = sqlio.read_sql_query(query, conn)
        except(Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            if conn:
                cursor.close()
                conn.close()
            return result      

## Make Graphs

In [ ]:
def week_histogram():
    db = Database()
    df = db.read_habits_week()
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values(by=["date"])
    df["day"] = df["date"].dt.day_name()
    df = df.groupby(by=["date", "day","name"]).count().reset_index().sort_values(by=["date"])

    fig = go.Figure()
    fig.add_trace(go.Histogram2d(
        x = df.day,
        y = df.name,
        texttemplate= "%{z}",
        colorscale='Tempo',
    ))
    fig.write_html("./templates/weekly/histogram.html")


In [ ]:
def week_sunburst():
    db = Database()
    df = db.read_habits_week()
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values(by=["date"])
    df["day"] = df["date"].dt.day_name()
    df = df.groupby(by=["name"]).count().reset_index()
    df = df[["name","id"]]
    minds = ["Class", "Programming", "Group Work", "Paper", "Atlantic", "WSJ", "Economist","Reading"]
    bodies = ["Yoga","Weight","Run","Walk", "Personal Train"]
    souls = ["Timer", "Guided", "Dharma Talk", "Group"]
    mind_count = {"name":"Mind", "id":0}
    body_count = {"name":"Body", "id":0}
    soul_count = {"name":"Soul", "id":0}
    for mind in minds:
        try:
            mind_count["id"]+= df[df.name==mind]["id"].values[0]
        except:
            df = df.append({"name":mind,"id":0}, ignore_index=True)
    for body in bodies:
        try:
            body_count["id"]+= df[df.name==body]["id"].values[0]
        except:
            df = df.append({"name":body,"id":0}, ignore_index=True)

    for soul in souls:
        try:
            soul_count["id"]+= df[df.name==soul]["id"].values[0]
        except:
            df = df.append({"name":soul,"id":0}, ignore_index=True)
    df = df.append(mind_count, ignore_index=True)
    df = df.append(body_count, ignore_index=True)
    df = df.append(soul_count, ignore_index=True)
    df = df.T
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    fig = go.Figure()
    fig.add_trace(go.Sunburst(
        labels=["All", "Mind", "Body", "Soul", "Class", "Programming", "Group Work", "Paper", "Atlantic", \
                "WSJ", "Economist","Reading", "Yoga","Weight","Run","Walk", "Personal Train", "Timer", "Guided", "Dharma Talk", "Group"],
        parents=["", "All", "All", "All", "Mind", "Mind", "Mind", "Mind", "Mind", "Mind", "Mind", "Mind",\
                 "Body","Body","Body","Body","Body","Soul","Soul","Soul","Soul"],
        values=[df["Mind"]["id"] + df["Body"]["id"] + df["Soul"]["id"],df["Mind"]["id"] ,df["Body"]["id"] ,df["Soul"]["id"] ,df["Class"]["id"] ,df["Programming"]["id"] ,df["Group Work"]["id"] ,df["Paper"]["id"] ,df["Atlantic"]["id"] , 
            df["WSJ"]["id"] ,df["Economist"]["id"] ,df["Reading"]["id"] ,df["Yoga"]["id"] ,df["Weight"]["id"] ,df["Run"]["id"],df["Personal Train"]["id"] ,df["Walk"]["id"] ,df["Timer"]["id"] ,df["Guided"]["id"] ,df["Dharma Talk"]["id"] ,df["Group"]["id"]],
        branchvalues="total",
    ))
    fig.write_html("./templates/weekly/sunburst.html")




In [19]:
def week_food():
    db = Database()
    df = db.read_food_week()
    df['start'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str) + ':00:00', format='%Y-%m-%d')
    df["start"] = df["start"].dt.date
    df['end'] = df['start'] + pd.Timedelta(days=1)
    
    
    df = df.sort_values(by=["date"])
    df = df.groupby(by=["start","end", "time"]).agg({'name': lambda x: "<br>".join(x.tolist()), "homecooked":"sum"}).reset_index()
    df['homecooked'] = df['homecooked'].astype(bool).apply(lambda x: False if x == 0 else True)
    df.time = df.time.astype(int)
    df = df.sort_values(by=["time"])
    
    df["name"] = "<b>" + df["name"]
    fig = px.timeline(df, x_start="start", x_end="end", y="time", color="homecooked", text="name")
    fig.update_yaxes(autorange="reversed")

    fig.update_layout(
        autosize=False,
        width=1600,
        height=600,
    )

    fig.update_traces(textposition='auto')
    fig.write_image("./templates/weekly/weekly_food.png")
    fig.write_html("./templates/weekly/weekly_food.html")


In [20]:
week_food()

# Make API

In [ ]:
app = Flask(__name__)
api = Api(app)
#nav = Navigation(app)

In [ ]:
@app.route('/')
def my_form():
    db = Database()
    df = db.read_habits().sort_values(by=["name"])
    return render_template('my-form.html', column_names = df.columns.values, row_data=list(df.values.tolist()), link_column ="id", zip=zip)

@app.route('/', methods=['POST'])
def my_form_post():
    db = Database()
    form = request.form
    if form["habit"] =="Eat":
        try:
            home_cooked = True if form["hcooked"] == "on" else False
        except:
            home_cooked = False
        db.insert_food(form["habit"], str(date.today()), str(form["hour"]), str(form["fname"]), str(form["size"]), home_cooked)
        
    
    else:
        try:
            if form["hour"] == "none":
                db.insert_habit(form["habit"], str(date.today()))
            else:
                db.insert_habit(form["habit"], str(date.today()), form["hour"])
        except:
            db.delete_habit(form["id"])
    #print(request.form)
    #text = request.form['text']
    #processed_text = text.upper()
    df = db.read_habits().sort_values(by=["name"])
    return render_template('my-form.html', column_names = df.columns.values, row_data=list(df.values.tolist()), link_column ="id", zip=zip)

@app.route('/display')
def histogram():
    week_histogram()
    week_sunburst()
    week_food()
    return render_template("display.html")
    #return render_template("/weekly/histogram.html")



In [ ]:
class Habit(Resource):
    def __init__(self):
        self.db = Database()
    
    def get(self):
        return {"test":"Woo"}
    def post(self):
        parser = reqparse.RequestParser()
        parser.add_argument('habitName', required = True)
        parser.add_argument('date', required = True)
        args = parser.parse_args()
        if "today" in args["date"]:
            #result = self.db.insert_habit(args["habitName"])
            None
        else: 
            #result = self.db.insert_habit(args["habitName"], args["date"])
            None
        result = "works"
        print("Inserting " + args["habitName"] + "  " +args["date"])
        print(result)
        if result is "duplicate":
            return jsonify(isError= True,
                    message= "Duplicate",
                    statusCode= 200,
                    data= args)
        return jsonify(isError= False,
                    message= "Success",
                    statusCode= 200,
                    data= args)
    
api.add_resource(Habit, '/habit')

<>:21: SyntaxWarning:

"is" with a literal. Did you mean "=="?

<>:21: SyntaxWarning:

"is" with a literal. Did you mean "=="?

C:\Users\zeusg\AppData\Local\Temp/ipykernel_9700/1779009016.py:21: SyntaxWarning:

"is" with a literal. Did you mean "=="?



In [ ]:
if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.86.25:5000/ (Press CTRL+C to quit)
